In [ ]:
import numpy as np
import os
import json
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import pipeline, set_seed
from transformers import BertTokenizer, Trainer, TrainingArguments
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, random_split
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split
import random
import time
import datetime
import pandas as pd
import copy

In [ ]:
# Training of other models follows same pipeline as below - simply load different dataset in this cell

# Note: 'dev' in variable name refers to test set
os.chdir('/content/drive/MyDrive/ML_Diss')
bert_train_data = list(np.load('bert_train_data.npy', allow_pickle = 'True'))
bert_dev_data = list(np.load('bert_dev_data.npy', allow_pickle = 'True'))

In [ ]:
# train hugging_face pretrained:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn as nn
import json

hg_model_hub_name = "facebook/bart-large-mnli"

tokenizer = AutoTokenizer.from_pretrained(hg_model_hub_name)

model = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)

In [ ]:
# ADDED:
train_sentences = [(entry['evidence'], entry['claim'])  for entry in bert_train_data]
train_labels = [0 if entry['label'] == 'SUPPORTS' else 1 if entry['label'] == 'REFUTES' else 2 for entry in bert_train_data]

dev_sentences = [(entry['evidence'], entry['claim']) for entry in bert_dev_data]
dev_labels = [0 if entry['label'] == 'SUPPORTS' else 1 if entry['label'] == 'REFUTES' else 2 for entry in bert_dev_data]

encoded_dict_train = list()
encoded_dict_dev = list()

# For every sentence...
for idx, sent in enumerate(train_sentences):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    if idx % 1000 == 0:
           print(idx)
    premise = sent[0]
    hypothesis = sent[1]
    encoded_dict_train.append(tokenizer.encode_plus(
                       premise, hypothesis, # Add '[CLS]' and '[SEP]'
                       truncation_strategy='only_first',          # Pad & truncate all sentences.
                   return_token_type_ids = True, pad_to_max_length = True, return_tensors = 'pt', return_attention_mask = True))
    


for idx, sent in enumerate(dev_sentences):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    if idx % 1000 == 0:
           print(idx)
    premise = sent[0]
    hypothesis = sent[1]
    encoded_dict_dev.append(tokenizer.encode_plus(
                       premise, hypothesis, # Add '[CLS]' and '[SEP]'
                        truncation_strategy='only_first',          # Pad & truncate all sentences.
                   return_token_type_ids = True, pad_to_max_length = True, return_tensors = 'pt', return_attention_mask = True))

In [ ]:
input_ids_train_baseline = torch.cat(input_ids_train_baseline, dim = 0)
attention_mask_train_baseline = torch.cat(attention_mask_train_baseline, dim = 0)
token_type_ids_train_baseline = torch.cat(token_type_ids_train_baseline, dim = 0)

input_ids_dev_baseline = torch.cat(input_ids_dev_baseline, dim = 0)
attention_mask_dev_baseline = torch.cat(attention_mask_dev_baseline, dim = 0)
token_type_ids_dev_baseline = torch.cat(token_type_ids_dev_baseline, dim = 0)


labels_train_baseline = torch.tensor(train_labels)
labels_dev_baseline = torch.tensor(dev_labels)


In [ ]:
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train_baseline, attention_mask_train_baseline, token_type_ids_train_baseline, labels_train_baseline)
val_dataset = TensorDataset(input_ids_dev_baseline, attention_mask_dev_baseline, token_type_ids_dev_baseline, labels_dev_baseline)






# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 10

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


In [ ]:
# Function to calculate the accuracy of our predictions vs labels

from sklearn.metrics import precision_recall_fscore_support


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


def compute_metrics(preds, labels):
    preds = np.argmax(preds, axis=1).flatten()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    return precision, recall, f1

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    model = model.to(device)
    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
  print('Only CPU available')
  device = torch.device('cpu')

In [ ]:
# define optimizers:

import torch.nn as nn
epochs = 5
accumulation_steps = 5

optimizer = AdamW(model.parameters(), lr=0.001, correct_bias=False)
total_steps = (len(train_sentences) * epochs)/accumulation_steps

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)


In [ ]:
import random
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []
epochs = 5
accumulation_steps = 5
# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0
    total_train_accuracy = 0
    total_f1 = 0
    total_precision = 0
    total_recall = 0 
    model.train()

    # For each batch of training data...
    
    for step, batch in enumerate(train_dataloader):

 
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: token_type_ids
        #   [3]: labels

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_token_type_ids = batch[2].to(device)
        b_labels = batch[3].to(device)

       
        model.zero_grad()        
        outputs = model(b_input_ids,
                  token_type_ids= b_token_type_ids,
                  attention_mask= b_input_mask, labels= b_labels
                  )
        loss = outputs[0]
        logits = outputs[1]
        loss = loss/accumulation_steps
        loss.backward()

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.



        # NEW: GRADIENT ACCUMULATION STEP
        if (step + 1)% accumulation_steps == 0:

           # Move logits and labels to CPU
           logits = logits.detach().cpu().numpy()
           label_ids = b_labels.to('cpu').numpy()
           total_train_loss += loss.item()
           total_train_accuracy += flat_accuracy(logits, label_ids)
           precision, recall, f1 = compute_metrics(logits, label_ids)
           total_precision += precision
           total_recall += recall
           total_f1 += f1
          

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
           torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
           optimizer.step()
           model.zero_grad()        
        # Update the learning rate.
           scheduler.step()
        
        

    # Calculate the average loss over all of the batches.
    avg_train_loss = accumulation_steps* total_train_loss / len(train_dataloader)  
    avg_train_accuracy = accumulation_steps*total_train_accuracy / len(train_dataloader)
    avg_train_F1 = accumulation_steps*total_f1/ len(train_dataloader)
    avg_train_precision = accumulation_steps*total_precision/ len(train_dataloader)
    avg_train_recall = accumulation_steps*total_recall/ len(train_dataloader)
    print("  Train Accuracy: {0:.2f}".format(avg_train_accuracy))

    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    # Tracking variables : ADDED EXTRA:
    total_f1 = 0
    total_precision = 0
    total_recall = 0 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for step, batch in enumerate(validation_dataloader):
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_token_type_ids = batch[2].to(device)
        b_labels = batch[3].to(device)
        
        print(b_input_ids.device, b_input_mask.device, b_token_type_ids.device, b_labels.device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = model(b_input_ids,
                  token_type_ids=b_token_type_ids,
                  attention_mask=b_input_mask,
                  labels=b_labels)
            loss = outputs[0]
            logits = outputs[1]
        
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        precision, recall, f1 = compute_metrics(logits, label_ids)
        total_precision += precision
        total_recall += recall
        total_f1 += f1

        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_f1 = total_f1/len(validation_dataloader)
    avg_val_precision = total_precision/len(validation_dataloader)   
    avg_val_recall = total_recall/len(validation_dataloader)
    print("  Val Accuracy: {0:.2f}".format(avg_val_accuracy))



    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'F1': avg_val_f1,
            'Precision':avg_val_precision,
            'Recall': avg_val_recall,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))